In [1]:
import pandas as pd
import pandas_gbq
import plotly.express as px
import pandas_gbq
import sys, requests
from pathlib import Path
from zipfile import ZipFile
import urllib
from PIL import Image
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option("styler.format.thousands", ",")

In [3]:
#Change the date to the previous day
query = """
SELECT *, "Regular Season" as Category FROM `perceptive-ivy-290216.nba_api_box_defensive_player.all_time_reg_season`
UNION ALL
SELECT *, "Playoffs" as Category FROM `perceptive-ivy-290216.nba_api_box_defensive_player.all_time_playoffs`
"""
project_id = "perceptive-ivy-290216"
df_bq_misc_player = pandas_gbq.read_gbq(query, project_id=project_id, dialect='standard')

Downloading: 100%|██████████|


In [4]:
#USE FOR ALL TIME DATA
df_bq3=df_bq_misc_player.copy()

#USE FOR SINGLE SEASON PLOTS
# current_year="2023-24"
# df_bq2=df_bq_reg_season[df_bq_reg_season["Season"]==current_year].copy()

df_bq3=df_bq3.fillna(0)
types="Regular Season"
# types="Playoffs"
df_bq2=df_bq3[df_bq3["Category"]==types]
df_bq2.head()

,GAME_ID,HOME_TEAM,AWAY_TEAM,TEAM,PLAYER_NAME,position,blocks,defensiveRebounds,matchupAssists,matchupFieldGoalPercentage,matchupFieldGoalsAttempted,matchupFieldGoalsMade,matchupMinutes,matchupThreePointerPercentage,matchupThreePointersAttempted,matchupThreePointersMade,matchupTurnovers,partialPossessions,playerPoints,steals,switchesOn,Season,Category
12311,0022301178,Minnesota Timberwolves,Atlanta Hawks,Minnesota Timberwolves,Jaden McDaniels,F,1,3,3,0.563,16,9,11:26,0.250,4,1,3,62.1,22,0,0,2023-24,Regular Season
12312,0022301178,Minnesota Timberwolves,Atlanta Hawks,Minnesota Timberwolves,Karl-Anthony Towns,F,1,4,4,0.467,15,7,10:33,0.286,7,2,1,55.1,19,0,0,2023-24,Regular Season
12313,0022301178,Minnesota Timberwolves,Atlanta Hawks,Minnesota Timberwolves,Rudy Gobert,C,5,15,3,0.407,27,11,14:29,0.250,4,1,1,76.0,25,1,0,2023-24,Regular Season
12314,0022301178,Minnesota Timberwolves,Atlanta Hawks,Minnesota Timberwolves,Anthony Edwards,G,0,4,2,0.571,7,4,10:39,0.000,3,0,1,55.0,9,2,0,2023-24,Regular Season
12315,0022301178,Minnesota Timberwolves,Atlanta Hawks,Minnesota Timberwolves,Mike Conley,G,0,5,5,0.571,7,4,11:38,0.000,1,0,1,63.6,11,2,0,2023-24,Regular Season


In [6]:
df_bq2.columns

Index(['GAME_ID', 'HOME_TEAM', 'AWAY_TEAM', 'TEAM', 'PLAYER_NAME', 'position',
       'blocks', 'defensiveRebounds', 'matchupAssists',
       'matchupFieldGoalPercentage', 'matchupFieldGoalsAttempted',
       'matchupFieldGoalsMade', 'matchupMinutes',
       'matchupThreePointerPercentage', 'matchupThreePointersAttempted',
       'matchupThreePointersMade', 'matchupTurnovers', 'partialPossessions',
       'playerPoints', 'steals', 'switchesOn', 'Season', 'Category'],
      dtype='object')

In [13]:
#Get Totals For Each Category
df_totals=pd.DataFrame(df_bq2.groupby(by=['PLAYER_NAME'])[[
       'blocks', 'defensiveRebounds', 'matchupAssists', 'matchupFieldGoalsAttempted',
       'matchupFieldGoalsMade', 'matchupThreePointersAttempted',
       'matchupThreePointersMade', 'matchupTurnovers',
       'playerPoints', 'steals', 'switchesOn']].sum()).reset_index()

df_totals['matchupFieldGoalPercentage']=df_totals['matchupFieldGoalsMade']/df_totals['matchupFieldGoalsAttempted']
df_totals['matchupThreePointerPercentage']=df_totals['matchupThreePointersMade']/df_totals['matchupThreePointersAttempted']

#Get Game Counts for Each Player
df_count=pd.DataFrame(df_bq2.groupby(by=['PLAYER_NAME'])[["GAME_ID"]].count()).reset_index()
df_count.columns=["PLAYER_NAME","GP"]
df_totals=df_totals.merge(df_count, left_on=["PLAYER_NAME"],right_on=["PLAYER_NAME"])
df_totals.head()

,PLAYER_NAME,blocks,defensiveRebounds,matchupAssists,matchupFieldGoalsAttempted,matchupFieldGoalsMade,matchupThreePointersAttempted,matchupThreePointersMade,matchupTurnovers,playerPoints,steals,switchesOn,matchupFieldGoalPercentage,matchupThreePointerPercentage,GP
0,A.J. Lawson,3,51,40,146,78,34,13,20,196,12,0,0.534247,0.382353,57
1,AJ Green,4,94,100,371,174,131,45,41,463,15,0,0.469003,0.343511,91
2,AJ Griffin,14,132,141,621,300,225,74,76,746,43,0,0.483092,0.328889,92
3,Aaron Brooks,0,10,23,61,33,31,15,16,102,6,0,0.540984,0.483871,31
4,Aaron Gordon,312,2267,1546,5325,2383,1747,599,724,5905,350,0,0.447512,0.342873,460


In [14]:
sort_category="matchupFieldGoalsMade"
# sort_category=[ 'blocks', 'defensiveRebounds', 'matchupAssists',
    #    'matchupFieldGoalPercentage', 'matchupFieldGoalsAttempted',
    #    'matchupFieldGoalsMade',
    #    'matchupThreePointerPercentage', 'matchupThreePointersAttempted',
    #    'matchupThreePointersMade', 'matchupTurnovers', 'partialPossessions',
    #    'playerPoints', 'steals', 'switchesOn']

In [15]:
#Sort by a category and then rank it. This will help get the Top Players in a category
df_totals['RK_Category']=df_totals[sort_category].rank(method="dense", ascending=False)
df_totals['RK_Category']=df_totals['RK_Category'].astype(int)
top=df_totals.sort_values(by=sort_category, ascending=False).head(10)
top

,PLAYER_NAME,blocks,defensiveRebounds,matchupAssists,matchupFieldGoalsAttempted,matchupFieldGoalsMade,matchupThreePointersAttempted,matchupThreePointersMade,matchupTurnovers,playerPoints,steals,switchesOn,matchupFieldGoalPercentage,matchupThreePointerPercentage,GP,RK_Category
884,Nikola Jokic,384,4572,2474,9906,4896,2212,805,1270,11308,678,0,0.494246,0.363924,521,1
887,Nikola Vucevic,430,4326,2294,8771,4349,1846,684,1128,10193,433,0,0.495839,0.370531,497,2
979,Rudy Gobert,1030,4562,2011,9956,4315,2097,707,1142,10107,351,0,0.433407,0.337148,485,3
121,Brook Lopez,938,1767,1936,9096,4109,2172,813,886,9716,255,0,0.451737,0.374309,462,4
295,Domantas Sabonis,239,4184,2067,7777,3762,1723,626,1123,9030,409,0,0.483734,0.36332,495,5
512,Jarrett Allen,627,3030,1773,7653,3573,1577,551,946,8384,294,0,0.466876,0.349398,484,6
569,Joel Embiid,635,3645,1625,7685,3464,1694,588,841,8247,365,0,0.450748,0.347107,398,7
587,Jonas Valanciunas,432,3616,1721,7159,3458,1476,542,836,8203,205,0,0.483028,0.367209,492,8
180,Clint Capela,716,3567,1771,6945,3328,1523,546,911,7753,320,0,0.479194,0.358503,453,9
128,Buddy Hield,195,1954,1574,6820,3328,2297,821,802,8228,505,0,0.487977,0.357423,548,9


In [16]:
#Get Player Names of the Top Category we have selected
names=top.PLAYER_NAME.to_list()

In [17]:
#Get the DF with Player Names from the top category
df_top=df_bq2[df_bq2['PLAYER_NAME'].isin(names)]
df_top.head()

,GAME_ID,HOME_TEAM,AWAY_TEAM,TEAM,PLAYER_NAME,position,blocks,defensiveRebounds,matchupAssists,matchupFieldGoalPercentage,matchupFieldGoalsAttempted,matchupFieldGoalsMade,matchupMinutes,matchupThreePointerPercentage,matchupThreePointersAttempted,matchupThreePointersMade,matchupTurnovers,partialPossessions,playerPoints,steals,switchesOn,Season,Category
12313,0022301178,Minnesota Timberwolves,Atlanta Hawks,Minnesota Timberwolves,Rudy Gobert,C,5,15,3,0.407,27,11,14:29,0.25,4,1,1,76.0,25,1,0,2023-24,Regular Season
12323,0022301178,Minnesota Timberwolves,Atlanta Hawks,Atlanta Hawks,Clint Capela,C,1,9,2,0.455,11,5,10:38,0.00,3,0,2,51.9,10,1,0,2023-24,Regular Season
12334,0022301194,Minnesota Timberwolves,Phoenix Suns,Minnesota Timberwolves,Rudy Gobert,C,1,4,11,0.455,11,5,14:33,0.00,3,0,3,63.8,11,2,0,2023-24,Regular Season
12370,0022200506,Washington Wizards,Philadelphia 76ers,Philadelphia 76ers,Joel Embiid,C,3,6,4,0.625,24,15,14:54,0.25,4,1,3,68.0,34,3,0,2022-23,Regular Season
12501,0022200412,Utah Jazz,New Orleans Pelicans,New Orleans Pelicans,Jonas Valanciunas,C,0,1,3,0.556,9,5,7:34,0.00,3,0,0,41.6,12,2,0,2022-23,Regular Season


In [18]:
#2 variable scatter plot

In [21]:
cat1="matchupFieldGoalsMade"
cat2="blocks"

fig_scatter = px.scatter(
  df_totals, 
  x=cat1, 
  y=cat2,
  color='PLAYER_NAME',
  color_discrete_sequence=px.colors.qualitative.G10,
  title="<b>{} All Time Player {} vs. {}</b>".format(types,cat1,cat2),
  template="plotly_white",
  hover_data=['blocks', 'defensiveRebounds', 'matchupAssists',
       'matchupFieldGoalPercentage', 'matchupFieldGoalsAttempted',
       'matchupFieldGoalsMade',
       'matchupThreePointerPercentage', 'matchupThreePointersAttempted',
       'matchupThreePointersMade', 'matchupTurnovers',
       'playerPoints', 'steals', 'switchesOn'], 
  height=800, 
  width=1200,
  )

fig_scatter.update_layout(
    title_x=0.5,
    hoverlabel=dict(
        bgcolor="white",
        font_size=16,
        font_family="PT Sans Narrow"
    ),
    yaxis = dict(tickfont = dict(size=20)),
    xaxis = dict(tickfont = dict(size=20)),
    font=dict(
        family="PT Sans Narrow",
        size=14,
        color="Black"
    ),
    title_font_family="PT Sans Narrow"
)
fig_scatter.update_traces(marker={'size': 10})

fig_scatter

In [25]:
fig_scatter.write_html("{} All Time Player {} vs. {}.html".format(types,cat1,cat2),full_html=False, include_plotlyjs='cdn')
# fig_scatter.write_html("Player {} vs. {} For The {} Season.html".format(cat1,cat2, current_year),full_html=False, include_plotlyjs='cdn')

In [ ]:
#Bar Chart For Category

In [23]:
fig_bar = px.bar(
  df_totals.sort_values(by=sort_category, ascending=False).head(25),
  x='PLAYER_NAME', 
  y=sort_category,
  color='PLAYER_NAME',
  color_discrete_sequence=px.colors.qualitative.G10,
  title="<b>{} All Time Player {}</b>".format(types,sort_category),
  template="plotly_white",
  hover_data=['blocks', 'defensiveRebounds', 'matchupAssists',
       'matchupFieldGoalPercentage', 'matchupFieldGoalsAttempted',
       'matchupFieldGoalsMade',
       'matchupThreePointerPercentage', 'matchupThreePointersAttempted',
       'matchupThreePointersMade', 'matchupTurnovers',
       'playerPoints', 'steals', 'switchesOn'], 
  height=800, 
  width=1200,
  )

fig_bar.update_layout(
    title_x=0.5,
    hoverlabel=dict(
        bgcolor="white",
        font_size=16,
        font_family="PT Sans Narrow"
    ),
    yaxis = dict(tickfont = dict(size=20)),
    xaxis = dict(tickfont = dict(size=20)),
    font=dict(
        family="PT Sans Narrow",
        size=14,
        color="Black"
    ),
    title_font_family="PT Sans Narrow"
)
fig_bar.update_traces(marker_color='#0047ab')
fig_bar

In [ ]:
fig_bar.write_html("{} All Time Player {}</b>".format(types,sort_category),full_html=False, include_plotlyjs='cdn')
# fig_bar.write_html("Player {} For The {} {}</b>".format(sort_category, current_year, types),full_html=False, include_plotlyjs='cdn')

In [ ]:
#MULTI METRIC PLOTS TOGETHER

In [27]:
sort_categories=[ 'blocks', 'defensiveRebounds', 'matchupAssists',
       'matchupFieldGoalPercentage', 'matchupFieldGoalsAttempted',
       'matchupFieldGoalsMade',
       'matchupThreePointerPercentage', 'matchupThreePointersAttempted',
       'matchupThreePointersMade', 'matchupTurnovers',
       'playerPoints', 'steals', 'switchesOn']

In [29]:
for i in sort_categories:
    df_totals['RK_Category']=df_totals[i].rank(method="dense", ascending=False)
    df_totals['RK_Category']=df_totals['RK_Category'].astype(int)
    top=df_totals.sort_values(by=i, ascending=False).head(10)
    names=top.PLAYER_NAME.to_list()
    df_top=df_bq2[df_bq2['PLAYER_NAME'].isin(names)]

    fig_bar_multi = px.bar(
    df_totals.sort_values(by=i, ascending=False).head(25),
    x='PLAYER_NAME', 
    y=i,
    color='PLAYER_NAME',
    color_discrete_sequence=px.colors.qualitative.G10,
    title="<b>{} All Time Player {}</b>".format(types,i),
    template="plotly_white",
  hover_data=['blocks', 'defensiveRebounds', 'matchupAssists',
       'matchupFieldGoalPercentage', 'matchupFieldGoalsAttempted',
       'matchupFieldGoalsMade',
       'matchupThreePointerPercentage', 'matchupThreePointersAttempted',
       'matchupThreePointersMade', 'matchupTurnovers',
       'playerPoints', 'steals', 'switchesOn'], 
    height=800, 
    width=1200,
    )

    fig_bar_multi.update_layout(
        title_x=0.5,
        hoverlabel=dict(
            bgcolor="white",
            font_size=16,
            font_family="PT Sans Narrow"
        ),
        yaxis = dict(tickfont = dict(size=20)),
        xaxis = dict(tickfont = dict(size=20)),
        font=dict(
            family="PT Sans Narrow",
            size=14,
            color="Black"
        ),
        title_font_family="PT Sans Narrow"
    )
    fig_bar_multi.update_traces(marker_color='#0047ab')
    fig_bar_multi.write_html("{} All Time Player {}.html".format(types,i),full_html=False, include_plotlyjs='cdn')
# fig_bar_multi.write_html("Player {} For The {} {}</b>".format(sort_category, current_year, types),full_html=False, include_plotlyjs='cdn')